In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import pyperclip
import pyautogui as pg
import time
import json
import os

In [ ]:

# Carrega o arquivo Excel
arquivo_excel = r"./agenda15_18.xlsx"

df = pd.read_excel(arquivo_excel)

# Caminho do arquivo JSON
json_file = 'ids_processados.json'

# Carrega os IDs processados do arquivo JSON, se ele existir
if os.path.exists(json_file):
    with open(json_file, 'r') as f:
        ids_processados = json.load(f)
else:
    ids_processados = []



In [3]:
# Configurar o ChromeDriver automaticamente
service = Service(ChromeDriverManager().install())

# Configuração do navegador (opcional: personalizar se necessário)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # Inicia o navegador maximizado

# Inicializar o WebDriver
driver = webdriver.Chrome(service=service, options=options) 

In [4]:
# Acessa a URL
driver.get('https://app10.ploomes.com/')

# Faz o login no Ploomes
time.sleep(1.3)
pg.write('email@email.com')
pg.press("tab")
pg.write('senha123')
pg.press('enter')
time.sleep(2)
time.sleep(10)

In [5]:
# Inicia um contador
cont = 1
roberto = 0
breno = 0
gisele = 0
rosangela = 0
pulados = 0

In [ ]:
driver.minimize_window()
driver.maximize_window()
start = time.time()
# Itera pelas linhas do DataFrame
for index, row in df.iterrows():
    # Armazena os valores em variáveis
    Id = row['NUREL']
    
    # Verifica se o ID já foi processado
    if Id in ids_processados:
        print(f'ID {Id} já processado. Pulando...')
        pulados +=1 
        continue
    
    Prioridade = row['CODHIST']
    Cliente = row['RAZAOSOCIAL']
    codParc = row['Cliente']
    HistCobr = row['Histórico de cobranças']
    HistVend = row['Histórico Vendas']
    Hist = row['Histórico']

    # Dividindo horário e data
    dtagend = row['Data de Agendamento']
    CodUsuario = row['CODUSU']
    dt = dtagend.split()
    # E utilizando apenas a data
    dtagend = dt[0]

    # Adicionando + 7 dias na data de agendamento do usuário 28
    if CodUsuario == 28:
        dtform = datetime.strptime(dtagend, "%d/%m/%Y")
        dtform += timedelta(days=7)
        dtagend = dtform.strftime("%d/%m/%Y")
    else:
        dtagend = dt[0]
    
    if CodUsuario == 28:
        Usu = 'BRENO'
        breno += 1
    elif CodUsuario == 12:
        Usu = 'R. VAIANO'
        rosangela += 1
    elif CodUsuario == 30:
        Usu = 'GISELE SOUZA'
        gisele += 1
    elif CodUsuario == 16:
        Usu = 'R. VAIANO'
        rosangela += 1
    elif CodUsuario == 6 or CodUsuario == 2 or CodUsuario == 13:
        continue
    else:
        # Dividindo as tarefas para que caso tenha alguma tarefa sem responsável seja enviada para o vendedor com menos agendamentos
        if rosangela > breno:
            Usu = 'Breno'
            breno += 1
        if breno >= rosangela:
            Usu = 'R. VAIANO'
            rosangela += 1
    
    # clica em novo
    driver.find_element('xpath', '//*[@id="react-root"]/div/div[2]/main-wrapper/app-wrapper/div/section/div/aside[2]/horizontal-menu/div/div[1]/button').click()
    time.sleep(0.35)
        
    # Clica em nova tarefa
    driver.find_element('xpath', '//*[@id="react-root"]/div/div[2]/main-wrapper/app-wrapper/div/section/div/aside[2]/horizontal-menu/div/div[3]/div[1]/div/div/button[1]').click()
    time.sleep(0.67)

    # Escreve o nome da tarefa
    driver.find_element("xpath", '//*[@id="new_task"]/div/ng-transclude/div[1]/ng-transclude/div/aside[1]/div[3]/section/forms/form/span[1]/div/div/div/new-field/span/input').send_keys(f'{codParc} - contatar {Cliente}')
    time.sleep(0.1)

    # Escreve a descrição da tarefa
    time.sleep(0.3)
    pyperclip.copy(f'*ID TELEMARKETING: {Id}*\n\nEntrar em contato com cliente {Cliente}')
    driver.find_element('xpath', '//*[@id="new_task"]/div/ng-transclude/div[1]/ng-transclude/div/aside[1]/div[3]/section/forms/form/span[2]/div/div/div/new-field/span/div/textarea').click()
    pg.hotkey('ctrl', 'v')
    time.sleep(0.1)

    # Limpa a data padrão e adiciona a data 
    driver.find_element("xpath",'//*[@id="datepicker_1003"]').clear()
    time.sleep(0.1)
    driver.find_element("xpath",'//*[@id="datepicker_1003"]').send_keys(f'{dtagend}')   
    time.sleep(0.5)
    pg.press('enter')

    # inicia um laço para encontrar o código do elemento 
    m = 0
    cont = cont+25
    while m == 0:
        try:
            # para cada número de 1 - infinito ele faz esse teste
            driver.find_element("xpath", f'//*[@id="select-fk-taskcontact-{cont}"]').send_keys(f'{codParc}')
            print(cont)
            time.sleep(1.8)
            pg.press('enter')
            pyperclip.copy(Cliente)
            print(Cliente)
            time.sleep(2)
            m = 2
        except:
            # Enquanto não encontrar o código ele aumenta em 1 e passa para o próximo
            cont += 1
            continue
    
    # A mesma teoria do código anterior
    m=0
    while m ==0:
        try:
            driver.find_element('xpath', f'//*[@id="select-fk-taskdeal-{cont}"]').send_keys(' ')
            time.sleep(0.9)
            pg.press('enter')
            time.sleep(0.3)
            try:
                time.sleep(0.1)
                driver.find_element('xpath', f'//*[@id="container-badges-taskdeal-{cont}"]/a/div/i')
                m=2
            except:
                try:
                    time.sleep(0.1)
                    driver.find_element('xpath', '/html/body/div[10]/div[2]/div/div/div[1]/div/div[2]').click()
                    time.sleep(0.2)
                    driver.find_element('xpath', '/html/body/div[10]/div[2]/div/div/forms/form/span[3]/div/div/div/new-field/span/select-fk/div/div/div/span/a/div/i').click()
                    time.sleep(0.2)
                    driver.find_element("xpath", '/html/body/div[10]/div[2]/div/div/forms/form/span[1]/div/div/div/new-field/span/input').send_keys('Oportunidade')
                    time.sleep(0.2)
                    if Usu == 'R. VAIANO' or CodUsuario == 16:
                        driver.find_element('xpath', '/html/body/div[10]/div[2]/div/div/forms/form/span[3]/div/div/div/new-field/span/select-fk/div/div/div/input[2]').send_keys('Roberto')
                    else:
                        driver.find_element('xpath', '/html/body/div[10]/div[2]/div/div/forms/form/span[3]/div/div/div/new-field/span/select-fk/div/div/div/input[2]').send_keys(Usu)
                    time.sleep(0.7)
                    pg.press('enter')
                    time.sleep(0.1)
                    driver.find_element('xpath', '/html/body/div[10]/div[2]/div/div/button[1]').click()
                    m = 2
                except:
                    m = 2        
        except:
            cont+=1
            continue
     
    # mesma teoria
    m = 0
    while m == 0:
            try:  
                # Nesse o código já começa com o valor anterior 
                time.sleep(0.1) 
                driver.find_element('xpath', f'//*[@id="container-badges-taskusers-{cont}"]/a/div/i').click()
                time.sleep(0.1)
                driver.find_element("xpath", f'//*[@id="select-fk-taskusers-{cont}"]').send_keys(Usu)
                print(cont)
                time.sleep(0.8)
                pg.press('enter')
                if CodUsuario == 12:
                    driver.find_element("xpath", f'//*[@id="select-fk-taskusers-{cont}"]').send_keys('Roberto')
                    time.sleep(1)
                    pg.press('enter')
                m = 2
            except:
                cont += 1
                continue

    # mesma teoria para adicionar o marcador
    m = 0
    while m == 0:
            try:  
                # Nesse o código já começa com o valor anterior  
                time.sleep(0.05)
                driver.find_element("xpath", f'//*[@id="select-fk-tasktags-{cont}"]').send_keys(Prioridade)
                print(cont)
                time.sleep(0.7)
                pg.press('enter')
                m = 2
            except:
                cont += 1
                continue

    time.sleep(3)
    # Volta para a tela do VSCODE
    pg.hotkey('alt', 'tab')
    # Input para aguardar o usuário conferir se todos itens estão corretos.
    e = input('Criar tarefa: ')
    # Caso o usuário digite 123 no input pulará a criação dessa tarefa
    if e == '123':
        continue
    pg.hotkey('alt', 'tab')
    time.sleep(0.1)
    driver.find_element("xpath", '//*[@id="new_task"]/div/ng-transclude/div[2]/div/div/ng-transclude/aside/button').click()
    time.sleep(0.55)

    # Adiciona o ID ao JSON
    ids_processados.append(Id)
    with open(json_file, 'w') as f:
        json.dump(ids_processados, f)

    # Mostra a quantidade de tarefas criadas
    print(f'{(index+1)-pulados} Tarefas criadas')

end = time.time()
total = end-start
print(f'{(index+1)-pulados} Tarefas criadas em {total:.2f}s\nTotalizando {(total/((index+1)-pulados)):.2f} segundos para cada tarefa.')
tpm = (60/(total/((index+1)-pulados)))
tph = tpm*60
print(f'Em um minuto serão criadas: {tpm:.0f} tarefas')
print(f'Em uma hora serão criadas: {tph:.0f} tarefas\n')
print(f'{breno} Tarefas cadastradas para o Breno\n{rob} Tarefas cadastradas para o Roberto\n{gisele} Tarefas cadastradas para a Gisele\n{dina} Tarefas cadastradas para a Diná\n{rosangela} Tarefas criadas para Rosangela/Roberto')

DUN DUN CHIPS PRODUTOS
63
DUN DUN CHIPS PRODUTOS ALIMENTICIOS LTDA                              
65
66
